# Download flickr dataset

In [ ]:
!wget https://github.com/xu-shitong/flickr8k-CLIP-freature/raw/master/image_all_final.pickle.zip
!wget https://github.com/xu-shitong/flickr8k-CLIP-freature/raw/master/text_all_final.pickle.zip
!unzip -q image_all_final.pickle.zip 
!unzip -q text_all_final.pickle.zip

# Imports

In [6]:
from PIL import Image
import pandas as pd
import copy
from torchvision.datasets import CocoCaptions
from transformers import (
  DistilBertTokenizer, DistilBertForMaskedLM, DistilBertConfig,
  CLIPProcessor, CLIPModel as CLIP, CLIPConfig
)
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import tqdm
import matplotlib.pyplot as plt
import os

if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
print("using device: ", dev)


/Users/xushitong/miniconda3/envs/mlenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using device:  cpu


In [ ]:
# download pretrained model and tokenizer
def save_model_tokenizer(tokenizer_class, model_class, name):
  if tokenizer_class is not None:
    tokenizer = tokenizer_class.from_pretrained(name)
    tokenizer.save_pretrained(f"./tokenizers/{name}-local")
  if model_class is not None:
    model = model_class.from_pretrained(name)
    model.save_pretrained(f"./models/{name}-local/")

save_model_tokenizer(CLIPProcessor, CLIP, "openai/clip-vit-base-patch32")


# Hyperparameters

In [62]:
# hyperparameters
BATCH_SIZE = 64
MAX_LENGTH = 64 # max text length
LEARNING_RATE = 5e-5
EPOCH_NUM = 1
ROUNDING_WEIGHT = 0.3 # weight of rounding term, the probability of regenerated sequence 

# diffusion hyperparameter
BETA_MIN = 0.0001
BETA_MAX = 0.02
STEP_TOT = 2000 # total noise adding steps
COSIN_SCHEDULE = True # if alpha sequence is scheduled in cosin instead of linear patten
SAMPLE_SIZE = 3 # number of sample steps in each diffuse sequence
X_0_PREDICTION = True # if model predicts x_0 or x_{t-1}

# Model, trainer and loss function

In [63]:
# TODO: change model as described in loss part
class DistilBertModel(nn.Module):
  def __init__(self, projection, config=None) -> None:
    '''
    inputs:
      projection: torch.tensor
      config
    '''
    super().__init__()

    self.model = DistilBertForMaskedLM(config).to(device)

    self.projection = nn.Linear(projection.shape[-2], projection.shape[-1], device=device)
    self.projection.weight.data = projection
    self.projection.bias.data = torch.zeros(self.projection.bias.data.shape, device=device)
    self.projection.requires_grad_(False)
    
    self.model.set_input_embeddings(nn.Sequential())
    self.model.set_output_embeddings(nn.Sequential())

  def parameters(self):
    return self.model.parameters()
  
  def forward(self, x, mask):
    '''
    return 
      feature_out, shape: [batch_size, seq_len, dim]
      vocab_out, shape: [batch_size, seq_len, vocab_size]
    '''
    
    x_out = self.model(x, mask)[0]
    return self.projection(x_out), x_out

clip_processor = CLIPProcessor.from_pretrained("./tokenizers/openai/clip-vit-base-patch32-local")
clip = CLIP.from_pretrained("./models/openai/clip-vit-base-patch32-local")

configuration = DistilBertConfig(vocab_size=clip_processor.tokenizer.vocab_size, dim=clip.projection_dim, n_heads=8)
model = DistilBertModel(clip.get_submodule("text_model.embeddings.token_embedding").weight.data, config=configuration)

# parameter only include model, no embedding layer
# trainer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
trainer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)


In [ ]:
if COSIN_SCHEDULE:
  def scheduler(t):
    s = 0.008 # smalle value prevent beta_t too small, from Improved DDPM paper
    return torch.cos(torch.pi / 2 * (t/STEP_TOT + s) / (1 + s)) ** 2
  ts = torch.arange(STEP_TOT).to(device)
  alpha_cumprod = scheduler(ts) / scheduler(torch.zeros(1, device=device))
else:
  betas = torch.hstack([torch.zeros(1), torch.linspace(BETA_MIN, BETA_MAX, STEP_TOT)]).to(device)
  alphas = 1 - betas
  alpha_cumprod = torch.cumprod(alphas[:-1], 0)
def diffuse_t(x, t):
  '''
  input:
    x_shape: [batch_size, seq_len, dim]
    t shape: [sample num] 
      NOTE: not necessary have hyperparameter sample_size number of element, to allow single diffuse generation

  return shape [sample_num * batch_size, seq_len, dim]
  '''
  batch_size, seq_len, dim = x.shape
  sample_shape = (t.numel(), *(1, ) * len(x.shape))

  noise = torch.normal(0, 1, x.shape).to(device)
  mean = torch.sqrt(alpha_cumprod[t].reshape(sample_shape)) * x 
  epsilon = noise * torch.sqrt(1 - alpha_cumprod[t]).reshape(sample_shape)
  return (mean + epsilon).reshape((t.numel() * batch_size, seq_len, dim))

def generate_diffuse_pair(x_0, t, t_next=None):
  '''
  input:
    x_0 shape: [batch_size, seq_len, dim],
    t shape: [sample_num] 
      NOTE: not necessary have hyperparameter sample_size number of element, to allow single diffuse generation
  
  return (net input, net target)
    net input shape: [sample_num * batch_size, seq_len, dim]
    net target shape: if t_next is None then [batch_size, seq_len, dim] else [sample_num * batch_size, seq_len, dim]
  '''
  if X_0_PREDICTION:
    # predict x_0
    return (diffuse_t(x_0, t), x_0)

  # predict x_{t_next}
  return (diffuse_t(x_0, t), diffuse_t(x_0, t_next))

def loss(model, image_clip, text_clip, x_t, x_1, x_tgt, x_0, mask, idx, loss_func):
  ''' 
  input: 
    model, input sequence: [x_t ... x_t, image_clip, text_clip], output at position [x_t ... x_t] is model prediction
    image_clip, text_clip shape: [batch_size, dim]
    x_t, x_tgt shape: [sample_num * batch_size, seq_len, dim]
      NOTE: x_tgt only used when X_0_PREDICTION is False
    x_1, x_0 shape: [batch_size, seq_len, dim]
    mask shape: [batch_size, seq_len]
    idx shape: [batch_size, seq_len]
    loss_func

  return triple loss terms
  '''
  
  repeat_shape = (SAMPLE_SIZE, *(1, ) * (len(x_t.shape) - 1))

  # x_t restore loss
  x_t_prob, x_t_hidden = model(x_t, mask.repeat(repeat_shape))
  if X_0_PREDICTION:
    x_t_loss = loss_func(x_t_hidden, x_0.repeat(repeat_shape))
  else:
    x_t_loss = loss_func(x_t_hidden, x_tgt)

  # x_1 restore loss
  x_1_prob, x_1_hidden = model(x_1, mask)
  x_1_loss = loss_func(x_1_hidden, x_0)

  # output sequence probability loss, applied to both x_1 and x_t restore
  idx = idx.unsqueeze(dim=-1)
  x_t_prob_loss = -(nn.functional.softmax(x_t_prob, dim=-1)).gather(-1, idx.repeat(repeat_shape)).log().mean()
  x_1_prob_loss = -(nn.functional.softmax(x_1_prob, dim=-1)).gather(-1, idx).log().mean()
  
  return x_t_loss, x_1_loss, ROUNDING_WEIGHT * (x_t_prob_loss + x_1_prob_loss)


# Define Dataset

In [85]:
class Flickr8kCLIPDataset(torch.utils.data.Dataset):
  def __init__(self, clip_processor) -> None:
    self.caption = pd.read_csv("flickr8k/captions.txt")
    self.tokenizer = clip_processor.tokenizer

    self.train_dataset = torch.utils.data.TensorDataset(torch.load("image_all_final.pickle"), torch.load("text_all_final.pickle"))

  def __len__(self):
    return len(self.caption)

  def __getitem__(self, idx):
    image_clip, text_clip = self.train_dataset[idx]
    embeddings = self.tokenizer(self.caption.loc[idx]["caption"], return_tensors="pt", padding='max_length', truncation=True, max_length=MAX_LENGTH)

    return image_clip, text_clip, embeddings["input_ids"], embeddings["attention_mask"]

# TODO: change to COCO dataset

train_dataset = Flickr8kCLIPDataset(clip_processor)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=BATCH_SIZE)


In [45]:
text = ["a photo of a cat", "a photo of a warship", "a photo of a boy", "a photo of a girl"]
image = Image.open("Shropshire.jpeg")
# plt.imshow(image)
# plt.show()
inputs = clip_processor(text=text, images=image, return_tensors="pt", padding=True)
inputs_no = clip_processor(text=text, images=None, return_tensors="pt", padding=True)

print(inputs.keys(), torch.all(inputs.input_ids == inputs_no.input_ids))

outputs = clip.get_text_features(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
# outputs = clip(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], pixel_values=torch.zeros((3,1,1)))

# outputs = clip(**inputs)
print(outputs.text_embeds.shape, outputs.image_embeds.shape, 
outputs.text_model_output["last_hidden_state"].shape, outputs.text_model_output["pooler_output"].shape, 
outputs.vision_model_output["last_hidden_state"].shape, outputs.vision_model_output["pooler_output"].shape, 
)
outputs.text_embeds, outputs.image_embeds, outputs.text_model_output, outputs.vision_model_output
# logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

# print(probs, text[probs.argmax(dim=-1)[0].item()])

dict_keys(['input_ids', 'attention_mask', 'pixel_values']) tensor(True)
tensor([[ 0.1555,  0.0733, -0.2448,  ..., -0.5327, -0.4588,  0.0346],
        [-0.0583,  0.1287,  0.1936,  ..., -0.2737,  0.0333, -0.3535],
        [-0.0936,  0.3672, -0.1155,  ..., -0.8218, -0.2455, -0.1423],
        [-0.1143,  0.2267, -0.2819,  ..., -0.6799, -0.2747,  0.1227]],
       grad_fn=<MmBackward0>)


ValueError: You have to specify pixel_values